# Seismic well tie

Let's make a synthetic with open source software! (And data!!)

This notebook uses `bruges`, `welly` (which uses `lasio`) and `segyio`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Load well logs: `welly` and `lasio`

We'll use `welly` to faciliate loading curves from an LAS file. Welly uses `lasio` to do the actual file reading.

Welly's `project` lets us load lots of wells. You can think of it like a list of wells with a few superpowers.

In [ ]:
from welly import Well, Project

base = "https://geocomp.s3.amazonaws.com/data/{}.las"

urls = [base.format(w) for w in ['R-39', 'L-30', 'R-90']]

wells = [Well.from_las(url) for url in urls]

p = Project(wells)

In [ ]:
p

In [ ]:
p[0]

In [ ]:
for w in p:
    w.header.uwi = w.header.name
p

## Choose a well and continue

In [ ]:
# Read one single well:
#    from welly import Well
#    l30 = Well.from_las('../data/L-30.las')

# But we have already loaded the well in the project.
# We can use its index...
#    l30 = p[2]
# ...or its UWI to get at it.
l30 = p.get_well('PENOBSCOT L-30')

In [ ]:
from welly import Well

l30 = Well.from_las('../data/L-30.las')

In [ ]:
l30.data["DT"]

In [ ]:
l30.data["RHOB"]

In [ ]:
fig, (ax0, ax1) = plt.subplots(ncols=2)
l30.data["RHOB"].plot(ax=ax0)
l30.data["DT"].plot(ax=ax1)

In [ ]:
dt = l30.data["DT"].top_and_tail()
rhob = l30.data["RHOB"].to_basis_like(dt)

In [ ]:
dt.units, dt.mnemonic, dt.start, dt.stop

In [ ]:
rhob.units, rhob.mnemonic, rhob.start, rhob.stop

### EXERCISE

- The density log is in g/cm³ — convert it to kg/m³.
- Retrieve the DT (slowness) log and convert it to μs/m.
- Compute a P-wave velocity (Vp) log in m/s from the slowness log.
- Compute the product of Vp and density to yield impedance.

In [ ]:
# YOUR CODE HERE



In [ ]:
dt /= 0.3048
dt.units = 'µs/m'

rhob *= 1000
rhob.units = 'kg/m3'

vp = 1e6 / dt
vp.mnemonic = 'VP'
vp.units = 'm/s'

ai = vp * rhob
ai.mnemonic = 'AI'
ai.units = 'Pa.s/m'  # Units of acoustic impedance for linear travel. "Viscosity per unit length"

In [ ]:
ai.plot()

In [ ]:
plt.figure(figsize=(16, 2))
plt.plot(ai.basis, ai, lw=0.5)

## Depth to time conversion: `numpy.interp()`

The logs are in depth, but the seismic is in travel time. So we need to convert the well data to time.

We don't know the seismic time, but we can model it from the DT curve: since DT is 'elapsed time', in microseconds per metre, we can just add up all these time intervals for 'total elapsed time'. Then we can use that to 'look up' the time of a given depth.

We use the step size to scale the DT values to 'seconds per step' (instead of µs/m).

We will need to know:

- The sample interval of the DT log.
- The well measurement datum.
- The ground level or water depth.
- The replacement velocity.

In [ ]:
l30.las.header['Well']['GL'].value

In [ ]:
rt = 0.3048 * l30.las.header['Well']['APDAT'].value  # This log is measured from RT not KB.
gl = 0.3048 * l30.las.header['Well']['GL'].value     # NB Before welly v 0.4.8 these were not captured.

rt, gl

In [ ]:
dt.start  # Relative to RT.

### EXERCISE

Do the arithmetic to find the timing of the top of the log. You need to know:

- Velocity is distance divided by time.
- The replacement velocity is unknown, use 1800 m/s for now.
- Use 1480 m/s as the velocity of water.
- Remember to multiply travel-times by 2 to get TWT.

You should get:

    Water time: 0.186 s
    Repl time:  0.204 s

In [ ]:
start =         # Start of DT log

v_water =       # Velocity of water
v_repl =        # Replacement velocity

water_layer =   # Depth of water
repl_layer =    # Thickness of replacement layer

water_twt =     # TWT in water, using water_layer and v_water
repl_twt =      # TWT in replacement layer, using repl_layer and v_repl

print(f"Water time: {water_twt:.3f} s\nRepl time:  {repl_twt:.3f} s")

In [ ]:
start = dt.start

v_water = 1480
v_repl = 1800

water_layer = -gl
repl_layer = start - water_layer - rt

water_twt = 2 * water_layer / v_water
repl_twt = 2 * repl_layer / v_repl

print(f"Water time: {water_twt:.3f} ms\nRepl time:  {repl_twt:.3f} ms")

Now we need to scale the DT log so that the samples represent 'elapsed time per sample':

In [ ]:
dt.step

In [ ]:
scaled_dt = 

In [ ]:
scaled_dt = dt.step * dt * 1e-6  # Convert to seconds per step
scaled_dt.units = "s/sample"

Now finally we can compute the cumulative time elapsed on the DT log.

This is the time-depth table.

In [ ]:
dt_time = 

In [ ]:
dt_time = water_twt + repl_twt + 2 * np.cumsum(scaled_dt)
dt_time.units = "s"

In [ ]:
dt_time.plot()

And then use this to convert the logs to a time basis:

In [ ]:
delt =             # Sample interval.
maxt =             # Max time that we need — longer than the log, and a multiple of delt.
n_samples =        # How many samples will that be?

seis_time = 

ai_t = np.interp()

In [ ]:
delt = 0.004                 # Sample interval.
maxt = np.ceil(dt_time[-1])  # Max time that we need; needs to be longer than the log.
n_samples = int(maxt / delt) + 1

# Make a regular time basis: the seismic time domain.
seis_time = np.linspace(0, maxt, n_samples) 

# OR...
# seis_time = np.arange(0, maxt, delt)

# Interpolate the AI log onto this basis.
ai_t = np.interp(seis_time, dt_time, ai)

We can also do this last step with `scipy`, which I prefer because (a) I prefer the API and (b) we have more options for interpolation algorithms (at least we do when we don't have a lot of NaNs in the data!):

In [ ]:
from scipy.interpolate import interp1d

f = interp1d(dt_time, ai, kind="slinear", bounds_error=False, fill_value="extrapolate")

ai_t_ = f(seis_time)

### EXERCISE

We'll turn all of this into functions.

- Make a time-conversion function to get time-converted logs from `delt`, `maxt`, `dt_time`, and a log.
- Make a function to get `dt_time` from `datum`, `gl`, `dt`, `v_water`, `v_repl`.
- Recompute `ai_t` by calling your new functions.
- Plot the DT log in time.

In [ ]:
def compute_dt_time(dt, datum, gl, v_repl, v_water=1480):
    """
    Compute DT time from the dt log and some other variables.
    
    The DT log must be a welly curve object.
    """

    # Your code here!
    
    return dt_time

In [ ]:
def compute_dt_time(dt, datum, gl, v_repl, v_water=1480):
    """
    Compute DT time from the dt log and some other variables.
    
    The DT log must be a welly curve object.
    """
    start = dt.start

    water_layer = -gl
    repl_layer = start - datum - water_layer
    
    water_twt = 2 * water_layer / v_water
    repl_twt = 2 * repl_layer / v_repl

    scaled_dt = dt.step * dt * 1e-6
    dt_time = water_twt + repl_twt + 2*np.cumsum(scaled_dt)
    dt_time.units = "s"

    return dt_time

In [ ]:
def time_convert(log, dt_time, delt=0.004, maxt=3.0):
    """
    Converts log to the time domain, given dt_time, delt, and maxt.
    
    dt_time is elapsed time regularly sampled in depth. log must
    be sampled on the same depth basis.
    """
    
    # Your code here!
    
    return log_t, seis_time  # Give the time basis back as well.

In [ ]:
def time_convert(log, dt_time, delt=0.004, maxt=None):
    """
    Converts log to the time domain, given dt_time, delt, and maxt.
    
    dt_time is elapsed time regularly sampled in depth. log must
    be sampled on the same depth basis.
    """
    maxt = maxt or np.ceil(dt_time[-1])
    t_seis = np.arange(0, maxt, delt)
    return np.interp(t_seis, dt_time, log), t_seis

Then these should work:

In [ ]:
dt_time = compute_dt_time(dt, rt, gl, v_repl=1800)
ai_t, t_seis = time_convert(ai, dt_time)

## Compute reflectivity

Now, at last, we can compute the reflection coefficients in time.

In [ ]:
def get_rc(ai):
    """
    Make reflections from impedance log.
    """
    rc = (ai[1:] - ai[:-1]) / (ai[1:] + ai[:-1])
    return np.pad(rc, (0, 1))

In [ ]:
rc = get_rc(ai_t)
rc[np.isnan(rc)] = 0

In [ ]:
plt.figure(figsize=(15, 2))
plt.stem(t_seis[600:700], rc[600:700], use_line_collection=True)

## Convolve with a wavelet: `bruges`

In [ ]:
from bruges.filters import ricker

w, t = ricker(0.128, 0.004, 20, return_t=True, sym=True)

In [ ]:
syn = np.convolve(rc, w, mode="same")

In [ ]:
plt.figure(figsize=(16,2))
plt.plot(t_seis, syn)

In [ ]:
start, stop = 220, 350
plt.figure(figsize=(16,4))
plt.plot(seis_time[start:stop], syn[start:stop], c='g', lw=2)

pts, stems, base = plt.stem(seis_time[start:stop], rc[start:stop], use_line_collection=True)
plt.setp(pts, markersize=5, c='r')
plt.setp(base, lw=0.75)

plt.show()

### EXERCISE

Make an interactive plot to allow us to vary the frequency of the wavelet.

In [ ]:
# YOUR CODE HERE



In [ ]:
from ipywidgets import interact

@interact(f=(4, 60, 4))
def makeplot(f):
    
    w, t = ricker(0.128, 0.004, f, return_t=True, sym=True)
    syn = np.convolve(rc, w, mode="same")

    start, stop = 250, 350
    plt.figure(figsize=(16,4))
    plt.plot(seis_time[start:stop], syn[start:stop], c='g', lw=2)

    pts, stems, base = plt.stem(seis_time[start:stop], rc[start:stop], use_line_collection=True)
    plt.setp(pts, markersize=5, c='r')
    plt.setp(base, lw=0.75)
    
    plt.xlabel('TWT [s]')
    plt.show()
    
    return

## Compare with the seismic: `segyio`

In [ ]:
import segyio

with segyio.open('../data/Penobscot_xl1155.sgy') as s:
    seismic = segyio.cube(s)[0]

The synthetic is at trace number 77. We need to make a shifted version of the synthetic to overplot.

In [ ]:
trace, gain = 77, 50
s = trace + gain*syn

And we can define semi-real-world cordinates of the seismic data:

In [ ]:
ma = np.percentile(seismic, 99)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(seismic.T, cmap='Greys', extent=(0, 400, 4.0, 0), aspect='auto', vmin=-ma, vmax=ma)
plt.plot(s, t_seis, c='cyan')
plt.fill_betweenx(t_seis, trace, s, where=syn>0, lw=0, color='cyan')
plt.xlim(0, 400)
plt.ylim(3.2, 0)
plt.show()

If we wanted to stretch the synthetic a little, the easiest thing to do is to create a new version of the DT log that we only use for time-keeping (you don't want time-based edits to be used for reflectivity). Then we can, for example, spread a time-shift at 2.5 s across the whole log.

In [ ]:
def compute_dt_time(dt, datum, gl, v_repl, v_water=1480, shift=None):
    """
    Compute DT time from the dt log and some other variables.
    
    The DT log must be a welly curve object.
    """
    start = dt.start

    water_layer = -gl
    repl_layer = start - datum - water_layer
    
    water_twt = 2 * water_layer / v_water
    repl_twt = 2 * repl_layer / v_repl

    if shift is not None:
        dt_corr = 1e6 * shift / dt.size 
    else:
        dt_corr = 0

    scaled_dt = dt.step * dt * 1e-6
    scaled_dt_corr = dt.step * (dt+dt_corr) * 1e-6
    dt_time = water_twt + repl_twt + 2 * np.cumsum(scaled_dt)
    dt_corr_time = water_twt + repl_twt + 2 * np.cumsum(scaled_dt_corr)
    dt_time.units = "s"

    return dt_time, dt_corr_time

In [ ]:
shift = 0.2   # seconds

dt_time, dt_corr = compute_dt_time(dt, rt, gl, v_repl=1800, shift=shift)
ai_t, t_seis = time_convert(ai, dt_corr)
dt_t, _ = time_convert(dt, dt_corr)
rho_t, _ = time_convert(rhob, dt_corr)

rc = get_rc(ai_t)
rc[np.isnan(rc)] = 0
syn = np.convolve(rc, w, mode="same")
s = trace + gain*syn

plt.figure(figsize=(10,10))
plt.imshow(seismic.T, cmap='Greys', extent=(0, 400, 4.0, 0), aspect='auto', vmin=-ma, vmax=ma)
plt.plot(s, t_seis, c='cyan')
# plt.plot(trace + rho_t*dt_t*0.00005, t_seis, c='green')
plt.fill_betweenx(t_seis, trace, s, where=syn>0, lw=0, color='cyan')
plt.xlim(0, 400)
plt.ylim(3.2, 0)
plt.colorbar()
plt.show()

In [ ]:
plt.plot(dt_time)
plt.plot(dt_corr)

In [ ]:
zz[-1] - dt_time[-1]

# This should be the time shift.

## Model a DTS log: `scikit` for now

We'd like to compute a gather, but this well doesn't have a shear sonic. Let's use another well to build a linear model from P-wave sonic and density.

First, we'll read data from another well and make our `X` matrix and and `y` vector:

In [ ]:
p

In [ ]:
r39 = p[0]

In [ ]:
dts = r39.data['DT4S']

The DTS has some problems (try plotting it!), so we'll fix those:

In [ ]:
dts[dts < 0] = np.nan
r39.data['DT4S'] = dts.interpolate()

In [ ]:
data = r39.data_as_matrix(keys=['RHOB', 'DT4P', 'DT4S'])
X = data[:, :2]
y = data[:, -1]

In [ ]:
plt.scatter(*data[:, :2].T)

Now we can select and fit a model.

In [ ]:
from sklearn.linear_model import Ridge

regr = Ridge().fit(X, y)

Make an `X` for application...

In [ ]:
l30.data['DT'].plot()

In [ ]:
X_appl = l30.data_as_matrix(keys=['RHOB', 'DT'])

...and apply the model to make a prediction for DTS:

In [ ]:
dts = regr.predict(np.nan_to_num(X_appl))

#### What do we think of this?

In [ ]:
dts

#### We'll have to go back and fix it.

Turn this into a curve.

In [ ]:
from welly import Curve

dts = Curve(dts, basis=l30.data['DT'].basis)

Fix some problems with bad (probably casing) values at the top and bottom:

In [ ]:
dts[dts < 100] = np.nan

In [ ]:
dts

## Backus averaging

Computing acoustic impedance this way is fine for a first pass, but eventually you'll want it to be more accurate. There's a couple of issues:

- It doesn't account for the limited seismic bandwidth.
- It doesn't account for anisotropy.
- It doesn't account for offset.

So let's employ Backus averaging, which gets at the first 2 points. According to Sherriff:

> An effective-medium theory used to upscale sonic-log data for synthetic seismogram manufacture. Involves harmonic averaging to find the anisotropic elastic parameters that characterize seismic-wave propagation at low frequencies in a layered medium.

Mavko suggests 10&times; the layer thickness (or beds in the formation) for the averaging length. So let's start with 10 m.

In [ ]:
import bruges as bg

vs = 1e6 / dts
vs.mnemonic = 'VS'
vs.units = 'm/s'

vp0, vs0, rho0 = bg.rockphysics.backus(vp, vs, rhob, lb=10, dz=0.1524)

In [ ]:
vp0.shape, vp.shape

In [ ]:
plt.plot(vp)
plt.plot(vp0)

## Compute offset gather

Now we can time-convert all the logs (before we just did the acoustic impedance log) and compute reflectivity.

In [ ]:
vp_t, _ = time_convert(vp0, dt_time)
vs_t, _ = time_convert(vs0, dt_time)
rhob_t, _ = time_convert(rho0, dt_time)

### EXERCISE

Use `bruges.reflection.reflectivity()`, which takes the logs we just made, to compute the offset-dependent reflectivity for, say, incident angles from 0 to 45 degrees.

You should end up with an `rc` array of shape (46, 749). Plot this array with `plt.imshow()`.

In [ ]:
# YOUR CODE HERE



In [ ]:
import bruges

rc = bruges.reflection.reflectivity(vp_t, vs_t, rhob_t, theta=np.linspace(0, 40, 41))

rc.shape

In [ ]:
rc_ = rc.T.real

plt.figure(figsize=(6, 15))
plt.imshow(rc_[250:500], aspect='auto')

In [ ]:
# To compute the intercept and gradient for this dataset...
I, G = bruges.reflection.shuey(vp_t[:-1], vs_t[:-1], rhob_t[:-1],
                               vp_t[1:], vs_t[1:], rhob_t[1:],
                               theta1=np.arange(0, 40, 1),
                               return_gradient=True
                              )

plt.figure(figsize=(6, 6))
plt.scatter(I, G, c=t_seis[:-1])
plt.axhline(0, c='k')
plt.axvline(0, c='k')
plt.axis('equal')
plt.grid()
plt.show()

In [ ]:
# To compute intercept and gradient from a real gather, we could do this...
I = (rc_[:, -1] - rc_[:, 0]) / np.sin(np.radians(40))**2
G = rc_[:, 0]

plt.figure(figsize=(6, 6))
plt.scatter(I, G, c=np.arange(rc_.shape[0]))
plt.axhline(0, c='k')
plt.axvline(0, c='k')
plt.axis('equal')
plt.grid()
plt.show()

### EXERCISE

Make a Ricker wavelet then use `np.apply_along_axis()` to make a 2D synthetic.

Finally, plot the result.

In [ ]:
# YOUR CODE HERE



In [ ]:
w = bruges.filters.ricker(0.256, 0.002, 20, sym=True)

In [ ]:
w = bruges.filters.ormsby(0.256, 0.002, (6, 12, 60, 80), sym=True)

In [ ]:
plt.plot(w)

In [ ]:
syn = np.apply_along_axis(np.convolve, axis=1, arr=rc, v=w, mode='same')

In [ ]:
plt.figure(figsize=(4, 10))
plt.imshow(syn.real.T, aspect='auto')


### EXERCISE

How does the zero-offset (aka normal incidence) synthetic compare to a simulation of the full stack (e.g. 0 to 30 degrees)?

In [ ]:
s = syn.real

plt.figure(figsize=(15, 3))
plt.plot(s[0])

fullstack = np.sum(s[:30], axis=0) / 30
plt.plot(fullstack)

plt.grid(c='k', alpha=0.15)

## Other things to try

- What difference does Backus averaging, or the improved wavelet, make to the tie quality?
- Can you export the synthetic or the gather as a LAS file (using `welly` or `lasio`), or as SEG-Y (using `segyio`)? 

<hr />

<div>
<img src="https://avatars1.githubusercontent.com/u/1692321?s=50"><p style="text-align:center">© Agile Geoscience 2020</p>
</div>